A Jupyter Notebook is an open source web application that allows you create and share cells of information.  These cells can include simple markdown, code, equations and more.  Notebooks can support a number of languages.  In this lab we'll be using a notebook which contains markdown syntax as well as Python code.

In a Jupyter Notebook, the area where text is entered and executed is called a cell.  You can create, edit and delete cells right within the notebook.  Visit this site for more information on [Jupyter Notebooks] (http://jupyter.org/)

PixieDust is an open source helper library for Python that can be added to a Jupyter Notebook in order to ease the complexity of working with data.  It allows users to import data and create data frames to manipulate the data into structures that make sense based on domain knowledge.  It also contains a power visualization API called display() that will allow us to display our data.  For more information on PixieDust go [here](https://github.com/pixiedust/pixiedust)

First we will import the PixieDust library into our notebook so that we can use the available API's.

In [ ]:
import pixiedust

In this lab we'll be using a data set which describes road safety data from the United Kingdom.  The statistics describe circumstances of personal injury and fatalities in accidents from 1979 until 2017.  Using this data set, we'll hope to obtain some interesting insights about road safety, accidents and how different aspects of the situation may influence outcomes.  The description of the data set and the accompanying files can be found here - [Road Safety Data](https://data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data)

First we'll use the Pixiedust API to import the data about the accidents.  Pixiedust requires a URL to a zip or CSV file to import it into a data frame.

In [ ]:
df_accidents = pixiedust.sampleData('http://data.dft.gov.uk/road-accidents-safety-data/dftRoadSafety_Accidents_2016.zip')

Next we'll use the API to import data about the vehicles involved in the accidents.

In [ ]:
df_vehicles = pixiedust.sampleData('http://data.dft.gov.uk/road-accidents-safety-data/MakeModel2016.zip')

The "df_accidents" and "df_vehicles" objects are data frames.  Data frames are two-dimensional array-like structures that contain columns and rows much like a database table.  They are a representation of the data and in our case contain all the available information that is contained in the imported data sets.  

These data sets are large and contain many columns or pieces of data that perhaps we are not interested in.  So now we can take the data frame, choose the information that we want from it and create a new view on the data.  This is done using Spark and SQL syntax.  The following cell will do the following:
- redefine the data frame choosing only the columns we want
- rename the columns if desired
- create a view for those column (Accidents) and print the new schema

In [ ]:
sqlContext.registerDataFrameAsTable(df_accidents, "Accidents")
df_accidents = sqlContext.sql("SELECT Accident_Index, Accident_Severity, `Number_of_Casualties` as Casualties, Day_of_Week, Time, Speed_limit FROM Accidents")
df_accidents.createOrReplaceTempView('Accidents')
df_accidents.printSchema()

We can also print the actual data in the view.  Since our data set is large we'll just print the first 10 rows.

In [ ]:
df_accidents.show(10)

Now let's reduce the columns in the "df_vehicles" data frame as well and show the new schema and the first 10 rows.  Again, this uses the Spark sqlContext which allows us to execute familiar SQL syntax.

In [ ]:
sqlContext.registerDataFrameAsTable(df_vehicles, "Vehicles")
df_vehicles = sqlContext.sql("SELECT Accident_Index, Vehicle_Type, `Age_Band_of_Driver` as Driver_Age, Make, Model FROM Vehicles")
df_vehicles.createOrReplaceTempView('Vehicles')
df_vehicles.printSchema()
df_vehicles.show(10)

Now we have two data frames with manageable columns.  They share a common key which is the column "Accident_Index".  We want to have this data in a single data frame for visualization.  In order to do this, we will combine these two data frames using a join operation on the common key.

In [ ]:
df_accidents_joined = df_accidents.join(df_vehicles, ['Accident_Index'], 'inner')
df_accidents_joined.printSchema()
df_accidents_joined.show(10)

Now we're going to talk about data refinement.  When a data scientist looks at real life data sets there may be pieces of data that don't make sense.  Perhaps there is data missing or in a format which is not understandable.  The data scientist would want to clean or refine the data in order to remove bad data or to transform it into something that is a little more usable.

Our data set has columns for Make and Model of the vehicle.  However in some cases, the Make and Model of the vehicle were not captured in the police report.  You may have noticed this in the rows that were printed above.  Since the data is not available, it is not going to help us in any prediction or insight scenario. Therefore we want to remove it.  In the next cell we will redefine the data frame again to remove any rows which don't contain valid Make or Model information.

In [ ]:
sqlContext.registerDataFrameAsTable(df_accidents_joined, "AccidentsCleaned")
df_accidents_cleaned = sqlContext.sql("SELECT * FROM AccidentsCleaned where Make NOT IN ('NULL', 'Missing') and Model NOT IN ('NULL', 'Missing') and Driver_Age <> -1 and Speed_limit <> 'NULL'")
df_accidents_cleaned.createOrReplaceTempView('AccidentsCleaned')
df_accidents_cleaned.printSchema()
df_accidents_cleaned.show(10)

Lastly we'll import some lookup data.  The format of the accident data in some cases uses integers to represent things like day of the week and age range.  Since we'd rather work with data that's more understandable we will import some lookup files to relate the integer values to labels that represent them.  This next cell will import lookup CSV files for day of the week and age range.

In [ ]:
df_day_lookup = pixiedust.sampleData('https://raw.githubusercontent.com/cloud-coder/cascon-2018-watson-studio/master/data/DayOfWeekLookup.csv')

In [ ]:
df_ageband_lookup = pixiedust.sampleData('https://raw.githubusercontent.com/cloud-coder/cascon-2018-watson-studio/master/data/AgeBandLookup.csv')

Now we'll use the lookup data frames and join them with our main data frame using SQL syntax.

In [ ]:
sqlContext.registerDataFrameAsTable(df_day_lookup, "DayOfWeekLookup")
sqlContext.registerDataFrameAsTable(df_ageband_lookup, "AgeBandLookup")
df_accidents_final = sqlContext.sql("SELECT ac.Accident_Index, ac.Accident_Severity, ac.Casualties, dl.Weekday, ac.Time, ac.Speed_limit, ac.Vehicle_Type, al.Age, ac.Make, ac.Model FROM AccidentsCleaned ac, DayOfWeekLookup dl, AgeBandLookup al where ac.Day_Of_Week=dl.code and ac.Driver_Age=al.code")
df_accidents_final.printSchema()

Now we have all our data imported, organized and refined.  Let's do some visualization!  

Pixiedust offers a visualization API which is as simple as using display() with the data frame as the input.  Once the data is displayed then you can work right within the user interface to manipulate what you want to see.  Start by just launching the display API on our final data frame.

In [ ]:
display(df_accidents_final)

The graphing options available here can be configured right within the display API.  First click on the Chart button (the chart icon with the arrow).  Choose a Bar Chart and try out the following graph:

1) Drag the Weekday column onto the Keys axis (X axis) if it's not already there.  
2) Drag the Casualties column onto the Values axis (Y axis) if it's not already there.  
3) Ensure that AVG is selected for the Aggregation type.  
4) Give the chart a title like "Casualties by Weekday."  
5) Click OK.  
6) After the initial graph renders, choose the drop down and select Pie Chart

This graph shows the number of casualties which occur based on the day of the week.  This shows that the average number of casualties stays relatively constant throughout the week and then increases on Saturday and Sunday.  

Go back to the chart options and change the Aggregation to COUNT.  Look at how the graph changes.  In fact, there are less casualties overall on Saturday and Sunday but as we saw, the average number is at it's highest on those days.  That makes sense given that there are fewer commuter cars on the road on the weekend and therefore less overall accidents and casualties.   

Try out another graph.  Click Options to adjust the information displayed in the graph.

1) Click Options on the left side of the chart.  
2) Drag the Speed_limit column onto the Keys axis (X axis) and remove the Weekday column.  
3) Drag the Casualtiies column onto the Values axis (Y axis).  
4) Ensure that AVG is selected for the Aggregation type.  
5) Give the chart a title like "Casualties by Speed Limit".  
6) Click OK.  

This graph now shows the average number of casualties (fatalities) which occur based on the road speed limit.  As one would expect, for accidents which occur on roads where the speed limit is higher, the average number of fatalities increases.

Spend some time playing around with the different types of graphing options and data within the graphing API.


This concludes this portion of the workshop which was intended to show you how an IBM Watson Studio user can use Notebooks to import, organize and graph data using Pixiedust.  A data scientist who is familiar with way of working may find this preferable.  However, for a non technical user it might seem complicated and non-intuitive.

In the next portion of the workshop, we'll be doing similar work and making similar observations using the IBM Watson Studio tooling.  This tooling might be more comfortable for a non technical user.